###### Introduction
This notebook is used for discovery of tenant wide artifacts, and will create/populate a specified Lakehouse.

Once the data has been saved to the lakehouse, a new semantic model and report will be created.

The process makes use of semantic link and semantic link labs.

###### Prerequisites
- Tenant admin will be required to identify all capacities, workspaces and items.
- XMLA Read/Write will need to be enabled on the capacity.

##### Install SemPy and Semantic Link Labs

In [1]:
%pip install semantic-link-labs

StatementMeta(, 12c38782-8d75-4ed4-a9ba-200636e78a8e, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.8/749.8 kB 8.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.10.2
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-5c37891a-5226-4218-912d-f7c02935aed8
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



Initialise Parameters

In [2]:
semantic_model_name = "sm_metadatascan_Trail_0825"
report_name = "rp_metadatascan_Trail_0825"
lakehouse = "lk_metadatascan_Trail_0825"

StatementMeta(, 12c38782-8d75-4ed4-a9ba-200636e78a8e, 9, Finished, Available, Finished)

##### Configuration - do not change.
Imports relevant libraries and semantic model and report jsonn


In [3]:
import sempy
import sempy.fabric as fabric
import sempy_labs as labs
import sempy_labs.admin as labs_admin
import sempy_labs.report as labs_report
import pandas as pd
import json

bimjson = {"compatibilityLevel":1604,"model":{"annotations":[{"name":"__PBI_TimeIntelligenceEnabled","value":"0"},{"name":"PBIDesktopVersion","value":"2.136.1202.0 (24.09)"},{"name":"PBI_QueryOrder","value":"[\"DatabaseQuery\"]"},{"name":"PBI_ProTooling","value":"[\"WebModelingEdit\",\"DevMode\",\"RemoteModeling\"]"},{"name":"TabularEditor_SerializeOptions","value":"{\"IgnoreInferredObjects\":True,\"IgnoreInferredProperties\":True,\"IgnoreTimestamps\":True,\"SplitMultilineStrings\":True,\"PrefixFilenames\":False,\"LocalTranslations\":True,\"LocalPerspectives\":True,\"LocalRelationships\":True,\"Levels\":[\"Data Sources\",\"Perspectives\",\"Relationships\",\"Roles\",\"Shared Expressions\",\"Tables\",\"Tables/Calculation Items\",\"Tables/Columns\",\"Tables/Hierarchies\",\"Tables/Measures\",\"Tables/Partitions\",\"Translations\"]}"},{"name":"__TEdtr","value":"1"}],"collation":"Latin1_General_100_BIN2_UTF8","culture":"en-US","cultures":[{"name":"en-US","linguisticMetadata":{"content":{"Language":"en-US","Version":"1.0.0"},"contentType":"json"}}],"dataAccessOptions":{"legacyRedirects":True,"returnErrorValuesAsNull":True},"defaultPowerBIDataSourceVersion":"powerBI_V3","expressions":[{"name":"DatabaseQuery","annotations":[{"name":"PBI_IncludeFutureArtifacts","value":"False"}],"expression":["let","\tdatabase = Sql.Database(\"tyar2jyz2t6ujk33yhkpavpd2q-bwp6bqqsncle7bvcpqieuwpnia.datawarehouse.fabric.microsoft.com\", \"d6d99fc2-872b-4040-b1d0-4a6a20ac7e82\")","in","\tdatabase"],"kind":"m","lineageTag":"9b99fd62-abc8-4b7b-9ebd-e696581f8fee"}],"relationships":[{"name":"4a857985-5fd6-ac96-f2cb-8af8dd37dd19","fromColumn":"Capacity_Id","fromTable":"Workspaces","relyOnReferentialIntegrity":True,"toColumn":"Capacity_Id","toTable":"Capacities"},{"name":"e18f9414-808d-1fbe-d784-ce5ff770c968","fromColumn":"Workspace_Id","fromTable":"WorkspaceItems","toColumn":"Id","toTable":"Workspaces"},{"name":"182b4873-f613-20e8-7c59-21afe30dda07","fromColumn":"Workspace_Id","fromTable":"SemanticModels","toColumn":"Id","toTable":"Workspaces"}],"sourceQueryCulture":"en-US","tables":[{"name":"Workspaces","annotations":[{"name":"PBI_ResultType","value":"Table"}],"columns":[{"name":"Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"f5095b8a-425a-4282-8906-4aa318949fdb","sourceColumn":"Id","sourceLineageTag":"Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Is_Read_Only","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"ac9e2faf-31a9-49e0-96f0-c1f875ade6e0","sourceColumn":"Is_Read_Only","sourceLineageTag":"Is_Read_Only","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Is_On_Dedicated_Capacity","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"c71e8a37-1588-4bfd-b4bb-6fbb1cdc1769","sourceColumn":"Is_On_Dedicated_Capacity","sourceLineageTag":"Is_On_Dedicated_Capacity","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Capacity_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"88954739-5272-4299-bef8-b206d2e0fed5","sourceColumn":"Capacity_Id","sourceLineageTag":"Capacity_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Default_Dataset_Storage_Format","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"088badad-570d-4b30-b589-855dc4cc9f52","sourceColumn":"Default_Dataset_Storage_Format","sourceLineageTag":"Default_Dataset_Storage_Format","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Type","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"fc3bc44b-8cd0-4aaa-9806-17be6b641f0d","sourceColumn":"Type","sourceLineageTag":"Type","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Name","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"c3d67c6f-7f0e-4bdb-8af9-38f97e000108","sourceColumn":"Name","sourceLineageTag":"Name","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Pipeline_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"3dacbf70-26d4-434c-a50c-09dc31432848","sourceColumn":"Pipeline_Id","sourceLineageTag":"Pipeline_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Has_Workspace_Level_Settings","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"739678b0-f57f-4972-9af9-ed8f3b2e49ba","sourceColumn":"Has_Workspace_Level_Settings","sourceLineageTag":"Has_Workspace_Level_Settings","sourceProviderType":"bit","summarizeBy":"none"},{"name":"State","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"49107ca3-67eb-4c85-bfa4-b1df21fee139","sourceColumn":"State","sourceLineageTag":"State","sourceProviderType":"varchar(8000)","summarizeBy":"none"}],"lineageTag":"dad1aa84-1ef4-49ae-901e-7dc8e29d3d7e","measures":[{"name":"Total Workspaces","changedProperties":[{"property":"Name"}],"expression":"COUNTROWS(Workspaces)","formatString":"0","lineageTag":"68a65d8d-7be5-4b0a-a0bd-93b13ee757c8"}],"partitions":[{"name":"Workspaces","mode":"directLake","source":{"entityName":"Workspaces","expressionSource":"DatabaseQuery","schemaName":"dbo","type":"entity"}}],"sourceLineageTag":"[dbo].[Workspaces]"},{"name":"WorkspaceItems","annotations":[{"name":"PBI_ResultType","value":"Table"}],"columns":[{"name":"Description","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"e6f20318-7265-4e2b-ac71-f1aa04a5661b","sourceColumn":"Description","sourceLineageTag":"Description","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Type","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"d03d5c35-f8c3-4294-a0ad-dc0754a2e0c9","sourceColumn":"Type","sourceLineageTag":"Type","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Workspace_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"0ad0d7ab-d488-4c71-8047-6e13c324fdf2","sourceColumn":"Workspace_Id","sourceLineageTag":"Workspace_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"State","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"936a6191-8e85-4f23-a8fb-3566ba0c72ef","sourceColumn":"State","sourceLineageTag":"State","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Last_Updated_Date","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"28159b9f-f957-4aad-b26d-6f8854e4cd6f","sourceColumn":"Last_Updated_Date","sourceLineageTag":"Last_Updated_Date","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Creator_Principal_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"4b830141-e37f-454f-81b1-cfdaf6dce6e3","sourceColumn":"Creator_Principal_Id","sourceLineageTag":"Creator_Principal_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Creator_Principal_Display_Name","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"656b2d1c-14f5-4099-ba4a-14bb5dff8d5f","sourceColumn":"Creator_Principal_Display_Name","sourceLineageTag":"Creator_Principal_Display_Name","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Creator_Principal_Type","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"5d0b44ac-312f-4371-a2a7-6843147153d5","sourceColumn":"Creator_Principal_Type","sourceLineageTag":"Creator_Principal_Type","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Capacity_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"57359b84-ff2d-4d16-83c2-cc102141686a","sourceColumn":"Capacity_Id","sourceLineageTag":"Capacity_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Item_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"42df17d6-87c6-4e05-abaa-8fdc5b9afbc4","sourceColumn":"Item_Id","sourceLineageTag":"Item_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Item_Name","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"1bb78e3d-df59-4688-8821-f16faab7768b","sourceColumn":"Item_Name","sourceLineageTag":"Item_Name","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Creator_User_Principal_Name","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"b5ac0d71-0230-41e5-86dc-9df89c4baa11","sourceColumn":"Creator_User_Principal_Name","sourceLineageTag":"Creator_User_Principal_Name","sourceProviderType":"varchar(8000)","summarizeBy":"none"}],"lineageTag":"02ff19b6-bacd-4c72-9518-e7315543550b","measures":[{"name":"Total Items","changedProperties":[{"property":"Name"}],"expression":"COUNTROWS(WorkspaceItems)","formatString":"0","lineageTag":"55325202-738c-41cc-874f-f7b09933ef55"},{"name":"Total PBI Items","changedProperties":[{"property":"Name"}],"expression":"CALCULATE(COUNTROWS(WorkspaceItems), WorkspaceItems[Type] IN { \"Dashboard\", \"PaginatedReport\", \"Datamart\", \"Report\" ,\"SemanticModel\" })","formatString":"0","lineageTag":"3e32c7a9-2855-4ba4-a8d0-5ed6d635bd0d"},{"name":"Total Non PBI Items","changedProperties":[{"property":"Name"}],"expression":"[Total Items] - [Total PBI Items]","formatString":"0","lineageTag":"ab55750d-350e-47fd-ae14-ba64376a5253"}],"partitions":[{"name":"WorkspaceItems","mode":"directLake","source":{"entityName":"WorkspaceItems","expressionSource":"DatabaseQuery","schemaName":"dbo","type":"entity"}}],"sourceLineageTag":"[dbo].[WorkspaceItems]"},{"name":"Capacities","annotations":[{"name":"PBI_ResultType","value":"Table"}],"columns":[{"name":"Capacity_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"b2f5f8ba-bcdf-4ae7-9756-fe851a67cac1","sourceColumn":"Capacity_Id","sourceLineageTag":"Capacity_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Capacity_Name","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"4c38d3e8-a3c0-4fc1-8b14-15731c0df200","sourceColumn":"Capacity_Name","sourceLineageTag":"Capacity_Name","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Sku","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"9c508b01-d10a-4a51-8c8d-8e5194a68c74","sourceColumn":"Sku","sourceLineageTag":"Sku","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Region","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"c810370b-e119-4e0c-9682-ff55c4114333","sourceColumn":"Region","sourceLineageTag":"Region","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"State","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"4279d1a0-c337-44bf-be87-430f4d156b1c","sourceColumn":"State","sourceLineageTag":"State","sourceProviderType":"varchar(8000)","summarizeBy":"none"}],"lineageTag":"6382af07-8377-43ab-97fa-5422e0da102a","measures":[{"name":"Total Capacities","changedProperties":[{"property":"Name"}],"expression":"COUNTROWS(Capacities)","formatString":"0","lineageTag":"0e26151c-257d-4051-8351-d593a6c52de0"},{"name":"Selected Capacity","changedProperties":[{"property":"Name"}],"expression":"SELECTEDVALUE(Capacities[Capacity_Name])","lineageTag":"9d57a129-d9a4-480d-89b6-9d0f5a86cb2b"},{"name":"Single Capacity Selected","changedProperties":[{"property":"Name"}],"expression":"IF(HASONEVALUE(Capacities[Capacity_Name]),\"\",\"Please select a single capacity to see size and region.\")","lineageTag":"66f870a5-16a5-40ed-81ad-37bbe53e2b91"},{"name":"Tansparent colour","changedProperties":[{"property":"Name"}],"expression":"IF(HASONEVALUE(Capacities[Capacity_Name]),\"#FFFFFF00\",\"#FFFFFF\")","lineageTag":"d9eb2daf-4dc3-429e-9545-fd53ea861dc8"}],"partitions":[{"name":"Capacities","mode":"directLake","source":{"entityName":"Capacities","expressionSource":"DatabaseQuery","schemaName":"dbo","type":"entity"}}],"sourceLineageTag":"[dbo].[Capacities]"},{"name":"SemanticModels","annotations":[{"name":"PBI_ResultType","value":"Table"}],"columns":[{"name":"Web_URL","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"28389d31-0303-4837-96ac-155309fe614d","sourceColumn":"Web_URL","sourceLineageTag":"Web_URL","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Add_Rows_API_Enabled","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"80cf5dc4-d579-4138-ad1f-7523f23581d3","sourceColumn":"Add_Rows_API_Enabled","sourceLineageTag":"Add_Rows_API_Enabled","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Configured_By","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"663da2f3-4c6b-441e-9275-0efe3fce2096","sourceColumn":"Configured_By","sourceLineageTag":"Configured_By","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Is_Refreshable","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"c21b47d5-edf1-4027-9a25-6453d5f3c6f0","sourceColumn":"Is_Refreshable","sourceLineageTag":"Is_Refreshable","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Is_Effective_Identity_Required","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"deb84945-981c-42f2-b711-12524dd6ad3d","sourceColumn":"Is_Effective_Identity_Required","sourceLineageTag":"Is_Effective_Identity_Required","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Is_Effective_Identity_Roles_Required","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"d5facb52-12b2-4c44-afd4-758bacf1b4d1","sourceColumn":"Is_Effective_Identity_Roles_Required","sourceLineageTag":"Is_Effective_Identity_Roles_Required","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Target_Storage_Mode","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"ce5c3c10-5673-4cde-a24c-ec8b2cf8f87b","sourceColumn":"Target_Storage_Mode","sourceLineageTag":"Target_Storage_Mode","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Created_Date","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"dateTime","formatString":"General Date","lineageTag":"9e64c6ff-711c-470a-802d-800f02552b3f","sourceColumn":"Created_Date","sourceLineageTag":"Created_Date","sourceProviderType":"datetime2","summarizeBy":"none"},{"name":"Content_Provider_Type","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"0336d188-fede-4aa9-a0de-e1bfb4281bbe","sourceColumn":"Content_Provider_Type","sourceLineageTag":"Content_Provider_Type","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Create_Report_Embed_URL","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"69796edd-fac1-4834-a17f-2a0b2644b1b3","sourceColumn":"Create_Report_Embed_URL","sourceLineageTag":"Create_Report_Embed_URL","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"QnA_Embed_URL","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"04766bb9-b2d6-406e-9167-2b8ba6b0920a","sourceColumn":"QnA_Embed_URL","sourceLineageTag":"QnA_Embed_URL","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Upstream_Datasets","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"8da30a86-e709-4ba9-b0e2-eed15cf45251","sourceColumn":"Upstream_Datasets","sourceLineageTag":"Upstream_Datasets","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Users","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"6da65361-030f-4904-88d8-0bc1af183e7c","sourceColumn":"Users","sourceLineageTag":"Users","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Is_In_Place_Sharing_Enabled","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"e3e025bb-2544-47be-89ee-a420b4dc3fa7","sourceColumn":"Is_In_Place_Sharing_Enabled","sourceLineageTag":"Is_In_Place_Sharing_Enabled","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Workspace_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"3196890f-bec4-45b3-ba39-db2bca6544c3","sourceColumn":"Workspace_Id","sourceLineageTag":"Workspace_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Dataset_Id","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"60a63951-4ff0-4d24-953d-206894b55ba7","sourceColumn":"Dataset_Id","sourceLineageTag":"Dataset_Id","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Dataset_Name","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"string","lineageTag":"905dd8e9-ed94-4cf5-9fcc-4be9a8bc3c83","sourceColumn":"Dataset_Name","sourceLineageTag":"Dataset_Name","sourceProviderType":"varchar(8000)","summarizeBy":"none"},{"name":"Auto_Sync_Read_Only_Replicas","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"boolean","formatString":"\"True\";\"True\";\"False\"","lineageTag":"8efcd80c-87fa-4fac-8b42-4c7499171e43","sourceColumn":"Auto_Sync_Read_Only_Replicas","sourceLineageTag":"Auto_Sync_Read_Only_Replicas","sourceProviderType":"bit","summarizeBy":"none"},{"name":"Max_Read_Only_Replicas","annotations":[{"name":"SummarizationSetBy","value":"Automatic"}],"dataType":"int64","formatString":"0","lineageTag":"3c3ff9ee-3149-47a6-8136-c6ffe1970bf9","sourceColumn":"Max_Read_Only_Replicas","sourceLineageTag":"Max_Read_Only_Replicas","sourceProviderType":"bigint","summarizeBy":"sum"}],"lineageTag":"fc54bd41-11d5-4d78-b14c-b1bcf0bc0b7f","measures":[{"name":"Large Models","changedProperties":[{"property":"Name"}],"expression":"CALCULATE(COUNTROWS(SemanticModels),SemanticModels[Target_Storage_Mode]=\"PremiumFiles\")","formatString":"0","lineageTag":"e900cf3c-d14b-41d4-9777-720fb72f4930"}],"partitions":[{"name":"SemanticModels","mode":"directLake","source":{"entityName":"SemanticModels","expressionSource":"DatabaseQuery","schemaName":"dbo","type":"entity"}}],"sourceLineageTag":"[dbo].[SemanticModels]"}]}}

reportjson = {"config": "{\"version\":\"5.57\",\"themeCollection\":{\"baseTheme\":{\"name\":\"CY24SU08\",\"version\":\"5.58\",\"type\":2}},\"activeSectionIndex\":0,\"defaultDrillFilterOtherVisuals\":true,\"linguisticSchemaSyncVersion\":0,\"settings\":{\"useNewFilterPaneExperience\":true,\"allowChangeFilterTypes\":true,\"useStylableVisualContainerHeader\":true,\"queryLimitOption\":6,\"useEnhancedTooltips\":true,\"exportDataMode\":1,\"useDefaultAggregateDisplayName\":true},\"objects\":{\"section\":[{\"properties\":{\"verticalAlignment\":{\"expr\":{\"Literal\":{\"Value\":\"'Top'\"}}}}}],\"outspacePane\":[{\"properties\":{\"expanded\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}]}}", "layoutOptimization": 0, "resourcePackages": [{"resourcePackage": {"disabled": False, "items": [{"name": "CY24SU08", "path": "BaseThemes/CY24SU08.json", "type": 202}], "name": "SharedResources", "type": 2}}], "sections": [{"config": "{\"relationships\":[{\"source\":\"0b6904bd7080047d5015\",\"target\":\"d04c601401d8226700c0\",\"type\":3},{\"source\":\"0b6904bd7080047d5015\",\"target\":\"51e498e51118b6e0e370\",\"type\":3},{\"source\":\"0b6904bd7080047d5015\",\"target\":\"4177ee3e6092e02b6310\",\"type\":3},{\"source\":\"0b6904bd7080047d5015\",\"target\":\"c29a0b57a3079307eb64\",\"type\":3},{\"source\":\"0b6904bd7080047d5015\",\"target\":\"ebf230576c6319ad159e\",\"type\":3},{\"source\":\"0b6904bd7080047d5015\",\"target\":\"af0710b041657d15c924\",\"type\":3},{\"source\":\"51e498e51118b6e0e370\",\"target\":\"4177ee3e6092e02b6310\",\"type\":3},{\"source\":\"51e498e51118b6e0e370\",\"target\":\"9cb6d7f92cabe239b500\",\"type\":3},{\"source\":\"51e498e51118b6e0e370\",\"target\":\"0e47247d7d08bb772270\",\"type\":3},{\"source\":\"51e498e51118b6e0e370\",\"target\":\"c29a0b57a3079307eb64\",\"type\":3},{\"source\":\"ebf230576c6319ad159e\",\"target\":\"4177ee3e6092e02b6310\",\"type\":3},{\"source\":\"ebf230576c6319ad159e\",\"target\":\"c29a0b57a3079307eb64\",\"type\":3},{\"source\":\"ebf230576c6319ad159e\",\"target\":\"0e47247d7d08bb772270\",\"type\":3},{\"source\":\"ebf230576c6319ad159e\",\"target\":\"9cb6d7f92cabe239b500\",\"type\":3},{\"source\":\"af0710b041657d15c924\",\"target\":\"9cb6d7f92cabe239b500\",\"type\":3},{\"source\":\"af0710b041657d15c924\",\"target\":\"0e47247d7d08bb772270\",\"type\":3},{\"source\":\"af0710b041657d15c924\",\"target\":\"4177ee3e6092e02b6310\",\"type\":3}],\"objects\":{\"background\":[{\"properties\":{\"image\":{\"image\":{\"name\":{\"expr\":{\"Literal\":{\"Value\":\"''\"}}},\"url\":{\"expr\":{\"Literal\":{\"Value\":\"''\"}}}}},\"transparency\":{\"expr\":{\"Literal\":{\"Value\":\"0D\"}}},\"color\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":0,\"Percent\":-0.1}}}}}}}]}}", "displayName": "Summary", "displayOption": 1, "filters": "[]", "height": 720.0, "name": "59c7a9391a0a15d0a7dd", "visualContainers": [{"config": "{\"name\":\"0078d9db84a234a36d01\",\"layouts\":[{\"id\":0,\"position\":{\"x\":13.617021276595745,\"y\":123.79110251450676,\"z\":1000,\"width\":1006.42166344294,\"height\":574.3907156673114,\"tabOrder\":9001}}],\"singleVisual\":{\"visualType\":\"shape\",\"drillFilterOtherVisuals\":true,\"objects\":{\"shape\":[{\"properties\":{\"tileShape\":{\"expr\":{\"Literal\":{\"Value\":\"'rectangle'\"}}},\"roundEdge\":{\"expr\":{\"Literal\":{\"Value\":\"8L\"}}}}}],\"rotation\":[{\"properties\":{\"shapeAngle\":{\"expr\":{\"Literal\":{\"Value\":\"0L\"}}}}}],\"fill\":[{\"properties\":{\"fillColor\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":0,\"Percent\":0}}}}}},\"selector\":{\"id\":\"default\"}}],\"outline\":[{\"properties\":{\"lineColor\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":2,\"Percent\":-0.5}}}}},\"weight\":{\"expr\":{\"Literal\":{\"Value\":\"2D\"}}}},\"selector\":{\"id\":\"default\"}}]},\"vcObjects\":{\"background\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}},\"transparency\":{\"expr\":{\"Literal\":{\"Value\":\"0D\"}}},\"color\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":0,\"Percent\":0}}}}}}}],\"border\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}],\"general\":[{\"properties\":{\"keepLayerOrder\":{\"expr\":{\"Literal\":{\"Value\":\"true\"}}}}}]}},\"howCreated\":\"InsertVisualButton\"}", "filters": "[]", "height": 574.39, "width": 1006.42, "x": 13.62, "y": 123.79, "z": 1000.0}, {"config": "{\"name\":\"0b6904bd7080047d5015\",\"layouts\":[{\"id\":0,\"position\":{\"x\":767.400468384075,\"y\":140.88992974238877,\"z\":9000,\"width\":242.81030444964873,\"height\":350.72599531615924,\"tabOrder\":7000}}],\"singleVisual\":{\"visualType\":\"multiRowCard\",\"projections\":{\"Values\":[{\"queryRef\":\"Capacities.Capacity_Name\"},{\"queryRef\":\"Capacities.Region\"},{\"queryRef\":\"Capacities.Sku\"},{\"queryRef\":\"Workspaces.Total Workspaces\"}]},\"prototypeQuery\":{\"Version\":2,\"From\":[{\"Name\":\"c\",\"Entity\":\"Capacities\",\"Type\":0},{\"Name\":\"w\",\"Entity\":\"Workspaces\",\"Type\":0}],\"Select\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Region\"},\"Name\":\"Capacities.Region\",\"NativeReferenceName\":\"Region\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Sku\"},\"Name\":\"Capacities.Sku\",\"NativeReferenceName\":\"Sku\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Capacity_Name\"},\"Name\":\"Capacities.Capacity_Name\",\"NativeReferenceName\":\"Capacity_Name\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w\"}},\"Property\":\"Total Workspaces\"},\"Name\":\"Workspaces.Total Workspaces\",\"NativeReferenceName\":\"Total Workspaces\"}],\"OrderBy\":[{\"Direction\":1,\"Expression\":{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Capacity_Name\"}}}]},\"drillFilterOtherVisuals\":true,\"hasDefaultSort\":true}}", "filters": "[]", "height": 350.73, "width": 242.81, "x": 767.4, "y": 140.89, "z": 9000.0}, {"config": "{\"name\":\"0e47247d7d08bb772270\",\"layouts\":[{\"id\":0,\"position\":{\"x\":1031.1798839458413,\"y\":9.90328820116054,\"z\":6000,\"width\":240.15473887814312,\"height\":382.5145067698259,\"tabOrder\":4000}}],\"singleVisual\":{\"visualType\":\"textbox\",\"drillFilterOtherVisuals\":true,\"objects\":{\"general\":[{\"properties\":{\"paragraphs\":[{\"textRuns\":[{\"value\":\"Report Summary\",\"textStyle\":{\"fontWeight\":\"bold\"}}]},{\"textRuns\":[{\"value\":\"This report is to allow tenant/capacity admins to review their current state in terms of Capacities prior to transitioning from Premium to Fabric Capacities.\"}]},{\"textRuns\":[{\"value\":\"\"}]},{\"textRuns\":[{\"value\":\"Transition Scenarios\",\"textStyle\":{\"fontWeight\":\"bold\"}}]},{\"textRuns\":[{\"value\":\"Same region\"}],\"listType\":\"bullet\"},{\"textRuns\":[{\"value\":\"Power BI Premium and Embedded supported\"}],\"listType\":\"bullet\"},{\"textRuns\":[{\"value\":\"Choose single, multi and all capacities\"}],\"listType\":\"bullet\"}]}}]},\"vcObjects\":{\"border\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"true\"}}},\"radius\":{\"expr\":{\"Literal\":{\"Value\":\"11D\"}}},\"color\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":2,\"Percent\":-0.5}}}}},\"width\":{\"expr\":{\"Literal\":{\"Value\":\"2D\"}}}}}]}}}", "filters": "[]", "height": 382.51, "width": 240.15, "x": 1031.18, "y": 9.9, "z": 6000.0}, {"config": "{\"name\":\"4177ee3e6092e02b6310\",\"layouts\":[{\"id\":0,\"position\":{\"x\":431.6627634660422,\"y\":23.981264637002344,\"z\":2000,\"width\":578.5480093676815,\"height\":65.94847775175644,\"tabOrder\":0}}],\"singleVisual\":{\"visualType\":\"multiRowCard\",\"projections\":{\"Values\":[{\"queryRef\":\"Capacities.Total Capacities\"},{\"queryRef\":\"Workspaces.Total Workspaces\"},{\"queryRef\":\"WorkspaceItems.Total Items\"}]},\"prototypeQuery\":{\"Version\":2,\"From\":[{\"Name\":\"c\",\"Entity\":\"Capacities\",\"Type\":0},{\"Name\":\"w\",\"Entity\":\"Workspaces\",\"Type\":0},{\"Name\":\"w1\",\"Entity\":\"WorkspaceItems\",\"Type\":0}],\"Select\":[{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Total Capacities\"},\"Name\":\"Capacities.Total Capacities\",\"NativeReferenceName\":\"Total Capacities\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w\"}},\"Property\":\"Total Workspaces\"},\"Name\":\"Workspaces.Total Workspaces\",\"NativeReferenceName\":\"Total Workspaces\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w1\"}},\"Property\":\"Total Items\"},\"Name\":\"WorkspaceItems.Total Items\",\"NativeReferenceName\":\"Total Items\"}],\"OrderBy\":[{\"Direction\":2,\"Expression\":{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Total Capacities\"}}}]},\"drillFilterOtherVisuals\":true,\"hasDefaultSort\":true,\"objects\":{\"card\":[{\"properties\":{\"outlineStyle\":{\"expr\":{\"Literal\":{\"Value\":\"0D\"}}}}}]},\"vcObjects\":{\"title\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}],\"visualHeader\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}],\"border\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}]}}}", "filters": "[]", "height": 65.95, "width": 578.55, "x": 431.66, "y": 23.98, "z": 2000.0}, {"config": "{\"name\":\"51e498e51118b6e0e370\",\"layouts\":[{\"id\":0,\"position\":{\"x\":37.30569948186529,\"y\":227.56476683937825,\"z\":4000,\"width\":730.259067357513,\"height\":264.87046632124355,\"tabOrder\":2000}}],\"singleVisual\":{\"visualType\":\"pivotTable\",\"projections\":{\"Rows\":[{\"queryRef\":\"Capacities.Capacity_Name\",\"active\":true},{\"queryRef\":\"Workspaces.Name\"}],\"Values\":[{\"queryRef\":\"Workspaces.Total Workspaces\"},{\"queryRef\":\"SemanticModels.Large Models\"},{\"queryRef\":\"WorkspaceItems.Total Items\"}]},\"prototypeQuery\":{\"Version\":2,\"From\":[{\"Name\":\"w\",\"Entity\":\"Workspaces\",\"Type\":0},{\"Name\":\"w2\",\"Entity\":\"WorkspaceItems\",\"Type\":0},{\"Name\":\"s\",\"Entity\":\"SemanticModels\",\"Type\":0},{\"Name\":\"c\",\"Entity\":\"Capacities\",\"Type\":0}],\"Select\":[{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w\"}},\"Property\":\"Total Workspaces\"},\"Name\":\"Workspaces.Total Workspaces\",\"NativeReferenceName\":\"Total Workspaces\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w2\"}},\"Property\":\"Total Items\"},\"Name\":\"WorkspaceItems.Total Items\",\"NativeReferenceName\":\"Total Items\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w\"}},\"Property\":\"Name\"},\"Name\":\"Workspaces.Name\",\"NativeReferenceName\":\"Name\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"s\"}},\"Property\":\"Large Models\"},\"Name\":\"SemanticModels.Large Models\",\"NativeReferenceName\":\"Large Models1\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Capacity_Name\"},\"Name\":\"Capacities.Capacity_Name\",\"NativeReferenceName\":\"First Capacity_Name\"}]},\"expansionStates\":[{\"roles\":[\"Rows\"],\"levels\":[{\"queryRefs\":[\"Capacities.Capacity_Name\"],\"isCollapsed\":true,\"identityKeys\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Entity\":\"Capacities\"}},\"Property\":\"Capacity_Name\"}}],\"isPinned\":true},{\"queryRefs\":[\"Workspaces.Name\"],\"isCollapsed\":true,\"identityKeys\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Entity\":\"Workspaces\"}},\"Property\":\"Name\"}}],\"isPinned\":true}],\"root\":{\"identityValues\":null}}],\"queryOptions\":{\"keepProjectionOrder\":true},\"drillFilterOtherVisuals\":true,\"objects\":{},\"vcObjects\":{\"title\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"true\"}}},\"text\":{\"expr\":{\"Literal\":{\"Value\":\"'Capacities'\"}}}}}],\"background\":[{\"properties\":{\"transparency\":{\"expr\":{\"Literal\":{\"Value\":\"100D\"}}}}}]}}}", "filters": "[]", "height": 264.87, "width": 730.26, "x": 37.31, "y": 227.56, "z": 4000.0}, {"config": "{\"name\":\"7e7d35d6b7f429a46a27\",\"layouts\":[{\"id\":0,\"position\":{\"x\":13.500082397963855,\"y\":10.500064087305221,\"z\":0,\"width\":1006.5061432259718,\"height\":102.00062256239356,\"tabOrder\":9002}}],\"singleVisual\":{\"visualType\":\"shape\",\"drillFilterOtherVisuals\":true,\"objects\":{\"shape\":[{\"properties\":{\"tileShape\":{\"expr\":{\"Literal\":{\"Value\":\"'rectangle'\"}}},\"roundEdge\":{\"expr\":{\"Literal\":{\"Value\":\"8L\"}}}}}],\"rotation\":[{\"properties\":{\"shapeAngle\":{\"expr\":{\"Literal\":{\"Value\":\"0L\"}}}}}],\"fill\":[{\"properties\":{\"fillColor\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":0,\"Percent\":0}}}}}},\"selector\":{\"id\":\"default\"}}],\"outline\":[{\"properties\":{\"lineColor\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":2,\"Percent\":-0.5}}}}},\"weight\":{\"expr\":{\"Literal\":{\"Value\":\"2D\"}}}},\"selector\":{\"id\":\"default\"}}]},\"vcObjects\":{\"background\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}},\"transparency\":{\"expr\":{\"Literal\":{\"Value\":\"0D\"}}},\"color\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":0,\"Percent\":0}}}}}}}],\"border\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}],\"general\":[{\"properties\":{\"keepLayerOrder\":{\"expr\":{\"Literal\":{\"Value\":\"true\"}}}}}]}},\"howCreated\":\"InsertVisualButton\"}", "filters": "[]", "height": 102.0, "width": 1006.51, "x": 13.5, "y": 10.5, "z": 0.0}, {"config": "{\"name\":\"9cb6d7f92cabe239b500\",\"layouts\":[{\"id\":0,\"position\":{\"x\":1031.1798839458413,\"y\":403.55899419729207,\"z\":7000,\"width\":240.15473887814312,\"height\":294.6228239845261,\"tabOrder\":5000}}],\"singleVisual\":{\"visualType\":\"textbox\",\"drillFilterOtherVisuals\":true,\"objects\":{\"general\":[{\"properties\":{\"paragraphs\":[{\"textRuns\":[{\"value\":\"\"}]}]}}]},\"vcObjects\":{\"border\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"true\"}}},\"color\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":2,\"Percent\":-0.5}}}}},\"radius\":{\"expr\":{\"Literal\":{\"Value\":\"10D\"}}},\"width\":{\"expr\":{\"Literal\":{\"Value\":\"2D\"}}}}}],\"background\":[{\"properties\":{\"color\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":0,\"Percent\":0}}}}}}}]}}}", "filters": "[]", "height": 294.62, "width": 240.15, "x": 1031.18, "y": 403.56, "z": 7000.0}, {"config": "{\"name\":\"af0710b041657d15c924\",\"layouts\":[{\"id\":0,\"position\":{\"x\":523.7209302325582,\"y\":501.3953488372093,\"z\":5000,\"width\":486.51162790697674,\"height\":176.74418604651163,\"tabOrder\":3000}}],\"singleVisual\":{\"visualType\":\"tableEx\",\"projections\":{\"Values\":[{\"queryRef\":\"SemanticModels.Dataset_Id\"},{\"queryRef\":\"SemanticModels.Dataset_Name\"},{\"queryRef\":\"SemanticModels.Target_Storage_Mode\"}]},\"prototypeQuery\":{\"Version\":2,\"From\":[{\"Name\":\"s\",\"Entity\":\"SemanticModels\",\"Type\":0}],\"Select\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"s\"}},\"Property\":\"Target_Storage_Mode\"},\"Name\":\"SemanticModels.Target_Storage_Mode\",\"NativeReferenceName\":\"Target_Storage_Mode1\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"s\"}},\"Property\":\"Dataset_Name\"},\"Name\":\"SemanticModels.Dataset_Name\",\"NativeReferenceName\":\"Dataset_Name\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"s\"}},\"Property\":\"Dataset_Id\"},\"Name\":\"SemanticModels.Dataset_Id\",\"NativeReferenceName\":\"Dataset_Id\"}]},\"drillFilterOtherVisuals\":true,\"objects\":{\"total\":[{\"properties\":{\"totals\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}]},\"vcObjects\":{\"title\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"true\"}}},\"text\":{\"expr\":{\"Literal\":{\"Value\":\"'Semantic Models'\"}}}}}],\"background\":[{\"properties\":{\"transparency\":{\"expr\":{\"Literal\":{\"Value\":\"100D\"}}}}}]}}}", "filters": "[]", "height": 176.74, "width": 486.51, "x": 523.72, "y": 501.4, "z": 5000.0}, {"config": "{\"name\":\"c29a0b57a3079307eb64\",\"layouts\":[{\"id\":0,\"position\":{\"x\":36.97111631537861,\"y\":27.978142076502735,\"z\":3000,\"width\":394.6916471506636,\"height\":56.95550351288056,\"tabOrder\":1000}}],\"singleVisual\":{\"visualType\":\"textbox\",\"drillFilterOtherVisuals\":true,\"objects\":{\"general\":[{\"properties\":{\"paragraphs\":[{\"textRuns\":[{\"value\":\"Summary\",\"textStyle\":{\"fontWeight\":\"bold\",\"fontSize\":\"24pt\"}}]}]}}]}}}", "filters": "[]", "height": 56.96, "width": 394.69, "x": 36.97, "y": 27.98, "z": 3000.0}, {"config": "{\"name\":\"cdda0b870bc1076a8686\",\"layouts\":[{\"id\":0,\"position\":{\"x\":37.30569948186529,\"y\":132.43523316062178,\"z\":11000,\"width\":729.3264248704663,\"height\":95.12953367875647,\"tabOrder\":9000}}],\"singleVisual\":{\"visualType\":\"slicer\",\"projections\":{\"Values\":[{\"queryRef\":\"Capacities.Capacity_Name\",\"active\":true}]},\"prototypeQuery\":{\"Version\":2,\"From\":[{\"Name\":\"c\",\"Entity\":\"Capacities\",\"Type\":0}],\"Select\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Capacity_Name\"},\"Name\":\"Capacities.Capacity_Name\",\"NativeReferenceName\":\"Capacity Name\"}]},\"columnProperties\":{\"Capacities.Capacity_Name\":{\"displayName\":\"Capacity Name\"}},\"drillFilterOtherVisuals\":true,\"objects\":{\"data\":[{\"properties\":{\"mode\":{\"expr\":{\"Literal\":{\"Value\":\"'Basic'\"}}}}}],\"general\":[{\"properties\":{\"orientation\":{\"expr\":{\"Literal\":{\"Value\":\"1D\"}}}}}]}}}", "filters": "[{\"expression\":{\"Column\":{\"Expression\":{\"SourceRef\":{\"Entity\":\"Capacities\"}},\"Property\":\"Capacity_Name\"}},\"filter\":{\"Version\":2,\"From\":[{\"Name\":\"c\",\"Entity\":\"Capacities\",\"Type\":0}],\"Where\":[{\"Condition\":{\"Not\":{\"Expression\":{\"Comparison\":{\"ComparisonKind\":0,\"Left\":{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Capacity_Name\"}},\"Right\":{\"Literal\":{\"Value\":\"null\"}}}}}}}]},\"type\":\"Advanced\",\"howCreated\":0,\"isHiddenInViewMode\":False}]", "height": 95.13, "width": 729.33, "x": 37.31, "y": 132.44, "z": 11000.0}, {"config": "{\"name\":\"d04c601401d8226700c0\",\"layouts\":[{\"id\":0,\"position\":{\"x\":767.400468384075,\"y\":140.88992974238877,\"z\":10000,\"width\":243.80952380952382,\"height\":350.72599531615924,\"tabOrder\":8000}}],\"singleVisual\":{\"visualType\":\"card\",\"projections\":{\"Values\":[{\"queryRef\":\"Capacities.Single Capacity Selected\"}]},\"prototypeQuery\":{\"Version\":2,\"From\":[{\"Name\":\"c\",\"Entity\":\"Capacities\",\"Type\":0}],\"Select\":[{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"c\"}},\"Property\":\"Single Capacity Selected\"},\"Name\":\"Capacities.Single Capacity Selected\",\"NativeReferenceName\":\"Single Capacity Selected\"}]},\"drillFilterOtherVisuals\":true,\"objects\":{\"categoryLabels\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"False\"}}}}}],\"labels\":[{\"properties\":{\"fontSize\":{\"expr\":{\"Literal\":{\"Value\":\"32D\"}}},\"color\":{\"solid\":{\"color\":{\"expr\":{\"ThemeDataColor\":{\"ColorId\":1,\"Percent\":0.6}}}}}}}]},\"vcObjects\":{\"background\":[{\"properties\":{\"transparency\":{\"expr\":{\"Literal\":{\"Value\":\"0D\"}}},\"color\":{\"solid\":{\"color\":{\"expr\":{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Entity\":\"Capacities\"}},\"Property\":\"Tansparent colour\"}}}}}}}]}}}", "filters": "[]", "height": 350.73, "width": 243.81, "x": 767.4, "y": 140.89, "z": 10000.0}, {"config": "{\"name\":\"ebf230576c6319ad159e\",\"layouts\":[{\"id\":0,\"position\":{\"x\":37.2093023255814,\"y\":501.3953488372093,\"z\":8000,\"width\":486.51162790697674,\"height\":176.74418604651163,\"tabOrder\":6000}}],\"singleVisual\":{\"visualType\":\"pivotTable\",\"projections\":{\"Rows\":[{\"queryRef\":\"WorkspaceItems.Type\",\"active\":true},{\"queryRef\":\"WorkspaceItems.Item_Name\"}],\"Values\":[{\"queryRef\":\"WorkspaceItems.Total Items\"}]},\"prototypeQuery\":{\"Version\":2,\"From\":[{\"Name\":\"w\",\"Entity\":\"WorkspaceItems\",\"Type\":0}],\"Select\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w\"}},\"Property\":\"Type\"},\"Name\":\"WorkspaceItems.Type\",\"NativeReferenceName\":\"Type\"},{\"Measure\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w\"}},\"Property\":\"Total Items\"},\"Name\":\"WorkspaceItems.Total Items\",\"NativeReferenceName\":\"Total Items\"},{\"Column\":{\"Expression\":{\"SourceRef\":{\"Source\":\"w\"}},\"Property\":\"Item_Name\"},\"Name\":\"WorkspaceItems.Item_Name\",\"NativeReferenceName\":\"Item_Name\"}]},\"expansionStates\":[{\"roles\":[\"Rows\"],\"levels\":[{\"queryRefs\":[\"WorkspaceItems.Type\"],\"isCollapsed\":true,\"identityKeys\":[{\"Column\":{\"Expression\":{\"SourceRef\":{\"Entity\":\"WorkspaceItems\"}},\"Property\":\"Type\"}}],\"isPinned\":true},{\"queryRefs\":[\"WorkspaceItems.Item_Name\"],\"isCollapsed\":true,\"isPinned\":true}],\"root\":{\"identityValues\":null}}],\"drillFilterOtherVisuals\":true,\"vcObjects\":{\"title\":[{\"properties\":{\"show\":{\"expr\":{\"Literal\":{\"Value\":\"true\"}}},\"text\":{\"expr\":{\"Literal\":{\"Value\":\"'Workspace Items'\"}}}}}],\"background\":[{\"properties\":{\"transparency\":{\"expr\":{\"Literal\":{\"Value\":\"100D\"}}}}}]}}}", "filters": "[]", "height": 176.74, "width": 486.51, "x": 37.21, "y": 501.4, "z": 8000.0}], "width": 1280.0}]}

StatementMeta(, 12c38782-8d75-4ed4-a9ba-200636e78a8e, 10, Finished, Available, Finished)

##### Setting up and Installing discovery report

In [ ]:
# Check if 'Lakehouse' exists otherwise create
df_items =fabric.list_items()
lakehouse_exists = df_items['Display Name'].eq(lakehouse).any()

if lakehouse_exists:
    print("Lakehouse already exists.")
    lhid = ( df_items[df_items['Display Name'].eq(lakehouse)&df_items['Type'].eq('Lakehouse')].iloc[0, 0])
else:
    print("Lakehouse does not exist. Creating...")
    lhid = fabric.create_lakehouse(lakehouse)


all_workspaces_df = labs_admin.list_workspaces(capacity="C89438CD-CFA3-4FF9-BE46-6F2F2E673CBF")
all_workspaces_df['Capacity Id'] = all_workspaces_df['Capacity Id'].fillna("-1")
labs.save_as_delta_table(dataframe=all_workspaces_df, delta_table_name="Workspaces", merge_schema = True ,write_mode="overwrite", lakehouse=lakehouse)

df_capacities = labs_admin._capacities._list_capacities_meta()

new_record = pd.DataFrame([{
    "Capacity Id": "-1",
    "Capacity Name": "Non Premium",
    "Sku": "N/A",
    "Region": "N/A",
    "State": "Active",
    "Admins": [["N/A"]]  # Must be a list
}])


df_capacities = pd.concat([df_capacities, new_record], ignore_index=True)
labs.save_as_delta_table(dataframe=df_capacities,delta_table_name="Capacities",merge_schema = True,write_mode="overwrite",lakehouse=lakehouse)

df_items = labs_admin.list_items(capacity="C89438CD-CFA3-4FF9-BE46-6F2F2E673CBF")
labs.save_as_delta_table(dataframe=df_items,delta_table_name="WorkspaceItems",merge_schema = True,write_mode="overwrite",lakehouse=lakehouse)


df_semantic_models = []

for workspace_id in all_workspaces_df["Id"]:
    filtercond = "workspaceId eq '" + workspace_id + "'"
    df_datasets=labs_admin.list_datasets(filter=filtercond)
    df_semantic_models.append(df_datasets)

df_semantic_models = pd.concat(df_semantic_models, ignore_index=True)


df_semantic_models["Upstream Datasets"] = df_semantic_models["Upstream Datasets"].astype(str)
df_semantic_models["Users"] = df_semantic_models["Users"].astype(str)
labs.save_as_delta_table(dataframe=df_semantic_models,delta_table_name="SemanticModels",merge_schema = True,write_mode="overwrite",lakehouse=lakehouse)

labs.create_semantic_model_from_bim(
    dataset=semantic_model_name,
    bim_file=bimjson)

In [ ]:
import time

check_interval = 5  # Interval in seconds between checks
max_retries = 12  # Maximum number of retries (e.g., 12 retries with 5 seconds interval = 1 minute)

# Loop to wait until there is a record in the DataFrame or max retries are reached
retries = 0
while retries < max_retries:
    df = fabric.list_datasets()
    df_filt = df[df["Dataset Name"] == semantic_model_name]
    
    if not df_filt.empty:
        labs.directlake.update_direct_lake_model_lakehouse_connection(dataset=semantic_model_name,lakehouse=lakehouse)

        labs_report.create_report_from_reportjson(
            report=report_name,
            dataset=semantic_model_name,
            report_json=reportjson
            )
        print('Report and Semantic Model created in current workspace.')
        break
    
    print(f"No model found. Retrying in {check_interval} seconds... ({retries + 1}/{max_retries})")
    time.sleep(check_interval)
    retries += 1

if retries == max_retries:
    print("Max retries reached. No semantic model found.")
